In [17]:
#!/usr/bin/env python3
import subprocess
import time
import psutil
from pathlib import Path

APP_PATH = "/Applications/Wondershare Filmora Mac.app"
APP_NAME = "Wondershare Filmora Mac"

def is_running() -> bool:
    for p in psutil.process_iter(attrs=["name"]):
        try:
            n = p.info.get("name") or ""
            if APP_NAME in n:
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    return False

def open_app():
    if not Path(APP_PATH).exists():
        raise FileNotFoundError(f"App not found at: {APP_PATH}")
    # Use `open` on the .app bundle
    subprocess.Popen(["open", APP_PATH])

def activate_and_fullscreen():
    applescript = f'''
    set appName to "{APP_NAME}"
    tell application appName to activate
    tell application "System Events"
        tell process appName
            set frontmost to true

            -- Wait up to ~15s for a window to appear
            repeat with i from 1 to 60
                if (count of windows) > 0 then exit repeat
                delay 0.25
            end repeat

            if (count of windows) > 0 then
                set theWindow to window 1

                -- Unminimize if needed
                try
                    if value of attribute "AXMinimized" of theWindow is true then ¬
                        set value of attribute "AXMinimized" of theWindow to false
                end try

                -- Prefer native macOS Full Screen if supported
                set didFS to false
                try
                    set value of attribute "AXFullScreen" of theWindow to true
                    set didFS to true
                end try

                if didFS is false then
                    -- Fallback: maximize to main display bounds
                    tell application "Finder" to set screenBounds to bounds of window of desktop
                    set screenWidth to item 3 of screenBounds
                    set screenHeight to item 4 of screenBounds
                    set position of theWindow to {{0, 0}}
                    set size of theWindow to {{screenWidth, screenHeight}}
                    try
                        set zoomed of theWindow to true
                    end try
                end if
            end if
        end tell
    end tell
    '''
    subprocess.run(["osascript", "-e", applescript], check=False)

def main():
    if not is_running():
        print("Filmora not running. Opening...")
        open_app()
        # Poll until the process appears (up to ~15s)
        for _ in range(60):
            if is_running():
                break
            time.sleep(0.25)
        time.sleep(4.0)  # small extra settle

    print("Focusing Filmora and full-screening…")
    activate_and_fullscreen()

if __name__ == "__main__":
    main()


Focusing Filmora and full-screening…


In [28]:
import pyautogui

pyautogui.moveTo(289, 126)

In [1]:
#Experimenting with pathfinding for import audio files 



In [2]:
def import_audio_clip(target_dir_audio: str, target_name_audio: str) -> None:
    pdir  = Path(target_dir_audio).expanduser().resolve()
    pfile = pdir / target_name_audio
    if not pfile.exists():
        raise FileNotFoundError(f"Audio file not found: {pfile}")

    ascript = f'''
    set appName    to "{APP_NAME}"
    set targetDir  to "{pdir.as_posix()}"
    set targetName to "{target_name_audio}"

    tell application appName to activate
    delay 0.3

    tell application "System Events"
        -- Ensure Filmora process exists and is frontmost
        repeat 60 times
            if (exists process appName) then exit repeat
            delay 0.1
        end repeat
        if (exists process appName) is false then return

        tell process appName
            set frontmost to true
            delay 0.1

            -- Open the Import dialog (supports both direct and nested menus)
            set fileMenu to menu "File" of menu bar 1
            set directItems to (menu items of fileMenu whose name contains "Import Media Files")
            if (count of directItems) > 0 then
                click item 1 of directItems
            else
                set parents to (menu items of fileMenu whose name contains "Import Media")
                if (count of parents) > 0 then
                    click (menu item 1 of (menu 1 of (item 1 of parents)))
                else
                    return
                end if
            end if

            -- Wait for the Open panel (as a sheet in fullscreen, or as a window)
            set thePanel to missing value
            repeat 100 times
                if (exists sheet 1 of window 1) then
                    set thePanel to sheet 1 of window 1
                    exit repeat
                else if (exists window 1 whose subrole is "AXDialog") then
                    set thePanel to (first window whose subrole is "AXDialog")
                    exit repeat
                end if
                delay 0.05
            end repeat
            if thePanel is missing value then return

            -- Bring focus to the panel and open 'Go to Folder…'
            set frontmost to true
            delay 0.1
            keystroke "g" using {{command down, shift down}}

            -- Wait for the small 'Go to the folder' sheet inside the panel
            set goSheet to missing value
            repeat 100 times
                if (exists sheet 1 of thePanel) then
                    set goSheet to sheet 1 of thePanel
                    exit repeat
                end if
                delay 0.05
            end repeat
            if goSheet is missing value then return

            -- Type the directory path precisely, then press Return
            -- (Direct value-setting can fail on some builds; keystroke is most compatible.)
            delay 0.1
            keystroke targetDir
            key code 36  -- Return

            -- Give Finder time to navigate the panel
            delay 0.2

            -- Now type the filename and press Return to open
            keystroke targetName
            key code 36  -- Return
        end tell
    end tell
    '''
    subprocess.run(["osascript", "-e", ascript], check=False)

In [9]:
from pathlib import Path
import subprocess
APP_NAME = "Wondershare Filmora Mac"
target_dir_audio, target_name_audio = "/Users/marcus/Movies/FilmoraInbox/reddit1_pipeline", "voice_20250902_234950.wav"
import_audio_clip(target_dir_audio, target_name_audio)